In [1]:
#!pip install seaborn

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

import numpy as np
import pandas as pd
from sklearn.datasets import load_boston #load boston data
import pickle

In [3]:
spark = SparkSession.builder.appName("SparkRegression").getOrCreate()

22/09/01 21:39:38 WARN Utils: Your hostname, Marios-MacBook-Pro-M1.local resolves to a loopback address: 127.0.0.1; using 172.20.10.4 instead (on interface en0)
22/09/01 21:39:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/01 21:39:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/01 21:39:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/01 21:39:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
# load data
boston_df = load_boston ()

/opt/homebrew/Caskroom/miniforge/base/envs/pydev_38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets inc

In [5]:

boston_df.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

In [6]:
#Check the description of the dataset
print(boston_df.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [7]:
# Gathering the features of the dataset with column names
df = pd.DataFrame(boston_df.data,columns=boston_df.feature_names)
#Creating a new column for the Target variable
df['PRICE'] = boston_df.target

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
#Converting DataFrame into a SparkDataframe
spark_df = spark.createDataFrame(df)

In [9]:
#Viewing the first 10 records of the SparkDataframe
spark_df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+-----+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|PRICE|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+-----+
|0.00632|18.0| 2.31| 0.0|0.538|6.575|65.2|  4.09|1.0|296.0|   15.3| 396.9| 4.98| 24.0|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421|78.9|4.9671|2.0|242.0|   17.8| 396.9| 9.14| 21.6|
|0.02729| 0.0| 7.07| 0.0|0.469|7.185|61.1|4.9671|2.0|242.0|   17.8|392.83| 4.03| 34.7|
|0.03237| 0.0| 2.18| 0.0|0.458|6.998|45.8|6.0622|3.0|222.0|   18.7|394.63| 2.94| 33.4|
|0.06905| 0.0| 2.18| 0.0|0.458|7.147|54.2|6.0622|3.0|222.0|   18.7| 396.9| 5.33| 36.2|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+-----+
only showing top 5 rows



In [10]:
#Understading the description of the Spark DataFrame
spark_df.printSchema()


root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: double (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: double (nullable = true)
 |-- TAX: double (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- PRICE: double (nullable = true)



In [11]:
#Quick summary of the stats of the data
spark_df.describe().show()

+-------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+
|summary|              CRIM|                ZN|             INDUS|               CHAS|                NOX|                RM|               AGE|               DIS|              RAD|               TAX|          PTRATIO|                B|             LSTAT|             PRICE|
+-------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+
|  count|               506|               506|               506|                506|                506|               506|               506|               506|            

## Data Checks and Cleaning

In [12]:
from pyspark.sql.functions import *

col_null_cnt_df =  spark_df.select([count(when(col(c).isNull(),c)).alias(c) for c in spark_df.columns])

print(col_null_cnt_df)

DataFrame[CRIM: bigint, ZN: bigint, INDUS: bigint, CHAS: bigint, NOX: bigint, RM: bigint, AGE: bigint, DIS: bigint, RAD: bigint, TAX: bigint, PTRATIO: bigint, B: bigint, LSTAT: bigint, PRICE: bigint]


## Data Transformation into Vectors

In [13]:
# Combining the features into a single Vector 
# In Sci-kit Learn each feature has its own column, and we don’t need to worry about applying 
# any more transformations to the dataframe. However, in MLlib, we cannot have individual columns for each feature. 
# MLlib requires all values of features in a row to be contained in an array, 
# and they all sit in one column called ‘features’.

# ********** IMPORTANT NOTE: Do not include any categorical column names whose values 
#     have not been StringIndexed or OneHotEncoded  **************

from pyspark.ml.feature import VectorAssembler

In [14]:
# Creating an instance of the VectorAssembler
assembler = VectorAssembler(inputCols=['CRIM','ZN','INDUS','CHAS',
                                       'NOX','RM','AGE','DIS','RAD',
                                       'TAX','PTRATIO','B','LSTAT'],
                            outputCol='features')

In [15]:
# transforming our spark dataframe
sparkdf_vector_out = assembler.transform(spark_df)
# Viewing the first 5 rows
sparkdf_vector_out.show(5)

22/09/01 21:39:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+-----+--------------------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|PRICE|            features|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+-----+--------------------+
|0.00632|18.0| 2.31| 0.0|0.538|6.575|65.2|  4.09|1.0|296.0|   15.3| 396.9| 4.98| 24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421|78.9|4.9671|2.0|242.0|   17.8| 396.9| 9.14| 21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07| 0.0|0.469|7.185|61.1|4.9671|2.0|242.0|   17.8|392.83| 4.03| 34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18| 0.0|0.458|6.998|45.8|6.0622|3.0|222.0|   18.7|394.63| 2.94| 33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18| 0.0|0.458|7.147|54.2|6.0622|3.0|222.0|   18.7| 396.9| 5.33| 36.2|[0.06905,0.0,2.18...|
+-------+----+-----+----+-----+-----+----+------+---+-----+-------+------+-----+-----+--------------------+
only showing top 5 rows



In [16]:
# While we could say something like this...
sparkdf_final_df = sparkdf_vector_out.select(['features', 'PRICE'])
# I want to rename the 'PRICE' column to 'label' as well
sparkdf_final_df = sparkdf_vector_out.select(['features', col('PRICE').alias('label')])
sparkdf_final_df.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.00632,18.0,2.3...| 24.0|
|[0.02731,0.0,7.07...| 21.6|
|[0.02729,0.0,7.07...| 34.7|
|[0.03237,0.0,2.18...| 33.4|
|[0.06905,0.0,2.18...| 36.2|
+--------------------+-----+
only showing top 5 rows



### Saving the Scaler Model

In [17]:
#Saving and serialising the assembler model
#pickle.dump('assembler',open('assembler.pkl','wb'))
#assembler.save("/Users/marionaidoo/Mario/GitHub/SparkRegressionModel_E2E/assembler")

In [18]:
#Correlation between the features (Vectored) 
Correlation.corr(sparkdf_final_df,'features').collect()

22/09/01 21:39:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/09/01 21:39:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
/opt/homebrew/Caskroom/miniforge/base/envs/pydev_38/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[Row(pearson(features)=DenseMatrix(13, 13, [1.0, -0.2005, 0.4066, -0.0559, 0.421, -0.2192, 0.3527, -0.3797, ..., -0.6138, 0.6023, -0.497, 0.4887, 0.544, 0.374, -0.3661, 1.0], False))]

## Creating the Train and Test Datasets

In [19]:
# Creating our train and test sets
train, test = sparkdf_final_df.randomSplit([0.7, 0.3], seed=42)

## Standardize Data sets

In [20]:
standard_scaler = StandardScaler(inputCol='features',outputCol='scaled')

In [21]:
train_scaled = standard_scaler.fit(train).transform(train)

In [22]:
test_scaled = standard_scaler.fit(test).transform(test)

### Saving the Scaler Model

In [23]:
#Saving and serialising the scaler model
#standard_scaler.save("/Users/marionaidoo/Mario/GitHub/SparkRegressionModel_E2E/std_scaler")

In [24]:
#Load std Scaler
#scaler_load = StandardScaler.load("/Users/marionaidoo/Mario/GitHub/SparkRegressionModel_E2E/std_scaler")

In [25]:
# #Saving and serialising the scaler model
# #pickle.dump(standard_scaler,open('std_scaler.pkl','wb'))

# ml_model_path = "/Users/marionaidoo/Mario/GitHub/SparkRegressionModel_E2E/" + 'std_scaler.pkl'

# with open(ml_model_path, 'wb') as fout:
#     pickle.dump(standard_scaler, fout)

In [26]:
train_scaled.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- scaled: vector (nullable = true)



## Regression Model

In [27]:
from pyspark.ml.regression import LinearRegression

# creating an instance of a linear regression model
lr_model = LinearRegression(featuresCol='scaled',labelCol='label')

In [28]:
# fitting the model to the train set
reg_model = lr_model.fit(train_scaled)

22/09/01 21:40:00 WARN Instrumentation: [f6d9f276] regParam is zero, which might cause numerical instability and overfitting.
22/09/01 21:40:00 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [29]:
## print the coefficients and the intercept
print(reg_model.coefficients)


[-1.0518503080747474,0.7870462635859095,0.405196033834234,1.187007129499578,-2.516618601676386,2.766281944250489,-0.3002551299576406,-3.134667763771697,2.647996542471487,-1.7305153149352868,-2.2437801773660544,0.9688770380130012,-3.5392280294200544]


In [30]:
print(reg_model.intercept)

36.78820776627808


## Test Results

In [31]:
test_results = reg_model.evaluate(test_scaled)

In [32]:
test_results.r2

0.45685298576485867

In [33]:
print(f"RMSE: {test_results.rootMeanSquaredError}")
print(f"R2: {test_results.r2}")
print(f"MeanSquaredError: {test_results.meanSquaredError}")

RMSE: 6.778796355461681
R2: 0.45685298576485867
MeanSquaredError: 45.952080028820575


## Saving the model

In [34]:
# import pickle
# pickle.dump(reg_model,open('regmodel.pkl','wb'))

import pickle
pickle_out = open("regmodel.pkl", "wb")
pickle.dump(reg_model, pickle_out)
pickle_out.close()

#reg_model.save("/Users/marionaidoo/Mario/GitHub/SparkRegressionModel_E2E/reg_model")

TypeError: cannot pickle '_thread.RLock' object